In [341]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.calibration import LabelEncoder

In [342]:
# -------------------------------
# 1. Load dataset
# -------------------------------
df = pd.read_csv('Data\k2pandc final.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4004 entries, 0 to 4003
Data columns (total 95 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   loc_rowid        4004 non-null   int64  
 1   pl_name          4004 non-null   object 
 2   hostname         4004 non-null   object 
 3   default_flag     4004 non-null   int64  
 4   disposition      4004 non-null   object 
 5   disp_refname     4004 non-null   object 
 6   sy_snum          4004 non-null   int64  
 7   sy_pnum          4004 non-null   int64  
 8   discoverymethod  4004 non-null   object 
 9   disc_year        4004 non-null   int64  
 10  disc_facility    4004 non-null   object 
 11  soltype          4004 non-null   object 
 12  pl_controv_flag  4004 non-null   int64  
 13  pl_refname       4004 non-null   object 
 14  pl_orbper        3960 non-null   float64
 15  pl_orbpererr1    3071 non-null   float64
 16  pl_orbpererr2    3071 non-null   float64
 17  pl_orbperlim  

In [343]:
# -------------------------------
# 2. ]Drop name and ID columns
# -------------------------------
cols_to_drop = [
    # IDs, references, dates
    'loc_rowid','pl_name','hostname','disp_refname','pl_refname','st_refname',
    'sy_refname','rowupdate','pl_pubdate','releasedate',
    
    # Orbital/planet mass errors and flags (keep main numeric columns)
    'pl_orbsmaxerr1','pl_orbsmaxerr2','pl_orbsmaxlim',
    'pl_bmasseerr1','pl_bmasseerr2','pl_bmasselim',
    'pl_bmassj','pl_bmassjerr1','pl_bmassjerr2','pl_bmassjlim','pl_bmassprov',
    'pl_orbeccen','pl_orbeccenerr1','pl_orbeccenerr2','pl_orbeccenlim',
    
    # Flux and temperature errors/limits (keep main numeric columns)
    'pl_insol','pl_insolerr1','pl_insolerr2','pl_insollim',
    'pl_eqterr1','pl_eqterr2','pl_eqtlim',
    
    # Stellar categorical / low coverage
    'st_spectype',
    
    # Stellar mass/metallicity errors & limits (keep main numeric columns)
    'st_masserr1','st_masserr2','st_masslim',
    'st_met','st_meterr1','st_meterr2','st_metlim','st_metratio',
    
    # Planet radius errors & limits
    'pl_radjerr1','pl_radjerr2','pl_radjlim',
    
    # Orbital period errors
    'pl_orbpererr1','pl_orbpererr2','pl_orbperlim',
    
    # Stellar teff & radius errors/limits
    'st_tefferr1','st_tefferr2','st_tefflim',
    'st_raderr1','st_raderr2','st_radlim',
    
    # Photometric errors
    'sy_vmagerr1','sy_vmagerr2','sy_kmagerr1','sy_kmagerr2',
    'sy_gaiamagerr1','sy_gaiamagerr2'
]

df = df.drop(columns=cols_to_drop)

In [344]:
# -------------------------------
# 2.1 Handle missing values (improved)
# -------------------------------
missing_pct = df.isnull().mean() * 100

# Drop columns with >50% missing values
cols_to_drop = missing_pct[missing_pct > 50].index
df = df.drop(columns=cols_to_drop)

# Update num_cols and cat_cols after dropping columns
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
cat_cols = df.select_dtypes(include=['object', 'category', 'bool']).columns

# Fill numerical columns with median
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Fill categorical columns with mode
for col in cat_cols:
    mode = df[col].mode()
    df[col] = df[col].fillna(mode[0] if not mode.empty else 'Unknown')

In [345]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4004 entries, 0 to 4003
Data columns (total 33 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   default_flag     4004 non-null   int64  
 1   disposition      4004 non-null   object 
 2   sy_snum          4004 non-null   int64  
 3   sy_pnum          4004 non-null   int64  
 4   discoverymethod  4004 non-null   object 
 5   disc_year        4004 non-null   int64  
 6   disc_facility    4004 non-null   object 
 7   soltype          4004 non-null   object 
 8   pl_controv_flag  4004 non-null   int64  
 9   pl_orbper        4004 non-null   float64
 10  pl_rade          4004 non-null   float64
 11  pl_radeerr1      4004 non-null   float64
 12  pl_radeerr2      4004 non-null   float64
 13  pl_radelim       4004 non-null   float64
 14  pl_radj          4004 non-null   float64
 15  ttv_flag         4004 non-null   int64  
 16  st_teff          4004 non-null   float64
 17  st_rad        

In [346]:
# -------------------------------
# 3. Scale numerical features
# -------------------------------
df_scaled = df.copy()
num_cols = df_scaled.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
df_scaled[num_cols] = scaler.fit_transform(df_scaled[num_cols])

In [347]:
# -------------------------------
# 4. Encode categorical features
# -------------------------------
##########
df_encoded = df_scaled.copy()
cat_cols = df_encoded.select_dtypes(include=['object', 'category', 'bool']).columns

le_dict = {}
for col in cat_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].fillna('NaN_Label'))
    le_dict[col] = le

In [348]:
# -------------------------------
# 5. Feature selection (correlation + importance)
# -------------------------------
from sklearn.ensemble import RandomForestClassifier

X = df_encoded.drop(columns=['disposition'])
y = df_encoded['disposition']

X = pd.get_dummies(X)  # one-hot if needed
X_train, _, y_train, _ = train_test_split(X, y, random_state=42)

# Correlation
corr_with_target = df_encoded.corr(numeric_only=True)['disposition'].drop('disposition').abs()
top_corr = corr_with_target.sort_values(ascending=False).head(7).index.tolist()

# Random Forest importance
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
top_importance = importances.head(7).index.tolist()

# Combine
selected_features = list(set(top_corr + top_importance))


In [349]:
# -------------------------------
# 6. Build preprocessed dataframe
# -------------------------------
X = df_encoded.drop(columns=['disposition'])
valuable_features_df = X[selected_features]

# Add target
preprocessed_df = valuable_features_df.copy()
preprocessed_df['disposition'] = df_encoded['disposition']


In [350]:
preprocessed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4004 entries, 0 to 4003
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sy_dist       4004 non-null   float64
 1   sy_vmag       4004 non-null   float64
 2   sy_pnum       4004 non-null   float64
 3   dec           4004 non-null   float64
 4   soltype       4004 non-null   int32  
 5   sy_kmag       4004 non-null   float64
 6   sy_disterr2   4004 non-null   float64
 7   default_flag  4004 non-null   float64
 8   pl_radeerr2   4004 non-null   float64
 9   disc_year     4004 non-null   float64
 10  sy_gaiamag    4004 non-null   float64
 11  disposition   4004 non-null   int32  
dtypes: float64(10), int32(2)
memory usage: 344.2 KB


In [351]:
preprocessed_df['disposition'].value_counts()

disposition
1    2315
0    1374
2     293
3      22
Name: count, dtype: int64

In [352]:
preprocessed_df = preprocessed_df[preprocessed_df['disposition'] != 3].reset_index(drop=True)

In [353]:
preprocessed_df['disposition'].value_counts()

disposition
1    2315
0    1374
2     293
Name: count, dtype: int64

In [354]:
from sklearn.model_selection import train_test_split

X = preprocessed_df.drop(columns=['disposition'])
y = preprocessed_df['disposition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [355]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Ensure y_train is numpy
if isinstance(y_train, torch.Tensor):
    y_train_np = y_train.cpu().numpy()
else:
    y_train_np = np.array(y_train)

# Get unique classes
classes = np.unique(y_train_np)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train_np
)

# Convert to tensor for PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

print("Classes:", classes)
print("Class Weights:", class_weights)


# ---- Focal Loss with class weights ----
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha  # can be class weights tensor
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)  # probability of true class
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss


# ---- Initialize with class weights ----
loss_fn = FocalLoss(alpha=class_weights, gamma=2.0)


Classes: [0 1 2]
Class Weights: tensor([0.9663, 0.5733, 4.5242], device='cuda:0')


In [356]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import recall_score, roc_auc_score
from sklearn.preprocessing import label_binarize

# Convert to tensors if not already
X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train = torch.tensor(y_train.to_numpy(), dtype=torch.long)
X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.long)

# Create datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Data loaders
batch_size = 32  # Adjust based on your data size and memory
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Compute class weights for imbalance (optional but recommended for recall)
class_counts = np.bincount(y_train.numpy())
class_weights = 1. / class_counts
class_weights = torch.tensor(class_weights / class_weights.sum(), dtype=torch.float32)  # Normalize

In [357]:
X_train.shape

torch.Size([2986, 11])

In [358]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_features, out_features, dropout_rate=0.3):  # <-- fixed __init__
        super(ResidualBlock, self).__init__()  # <-- fixed __init__
        self.fc = nn.Linear(in_features, out_features)
        self.ln = nn.LayerNorm(out_features)  # Layer norm for better stability
        self.dropout = nn.Dropout(dropout_rate)
        
        # Projection if dimensions differ
        self.proj = nn.Linear(in_features, out_features) if in_features != out_features else nn.Identity()
    
    def forward(self, x):
        residual = self.proj(x)
        x = F.silu(self.ln(self.fc(x)))  # Swish activation
        x = self.dropout(x)
        return x + residual  # Skip connection


class ResidualMLP(nn.Module):
    def __init__(self, input_size=16, num_classes=3):  # <-- fixed __init__
        super(ResidualMLP, self).__init__()  # <-- fixed __init__
        self.entry = nn.Linear(input_size, 256)  # Entry layer
        
        # Residual blocks for depth
        self.res_block1 = ResidualBlock(256, 256)
        self.res_block2 = ResidualBlock(256, 128)
        self.res_block3 = ResidualBlock(128, 128)
        self.res_block4 = ResidualBlock(128, 64)
        
        self.dropout_final = nn.Dropout(0.2)
        self.fc_out = nn.Linear(64, num_classes)
    
    def forward(self, x):
        x = F.silu(self.entry(x))
        
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        
        x = self.dropout_final(x)
        x = self.fc_out(x)  # Logits output
        return x


In [359]:
classes = np.unique(y_train)  # unique class labels
print("Classes:", classes)

Classes: [0 1 2]


In [360]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Ensure y_train is numpy
if isinstance(y_train, torch.Tensor):
    y_train_np = y_train.cpu().numpy()
else:
    y_train_np = np.array(y_train)

# Get unique classes
classes = np.unique(y_train_np)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train_np
)

# Convert to tensor for PyTorch
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

print("Classes:", classes)
print("Class Weights:", class_weights)


# ---- Focal Loss with class weights ----
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha  # can be class weights tensor
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)  # probability of true class
        focal_loss = (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        return focal_loss


# ---- Initialize with class weights ----
loss_fn = FocalLoss(alpha=class_weights, gamma=2.0)


Classes: [0 1 2]
Class Weights: tensor([0.9663, 0.5733, 4.5242], device='cuda:0')


In [361]:
print("CUDA Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("Number of GPUs:", torch.cuda.device_count())
    print("Current Device Index:", torch.cuda.current_device())

CUDA Available: True
GPU Name: NVIDIA GeForce RTX 4070 Laptop GPU
Number of GPUs: 1
Current Device Index: 0


In [362]:
X

,sy_dist,sy_vmag,sy_pnum,dec,soltype,sy_kmag,sy_disterr2,default_flag,pl_radeerr2,disc_year,sy_gaiamag
0,-0.387765,-1.229216,-0.091546,1.274274,1,-0.953914,0.188935,-0.906452,0.152988,-0.761754,-1.142112
1,-0.387765,-1.229216,-0.091546,1.274274,1,-0.953914,0.188935,-0.906452,0.157520,-0.761754,-1.142112
2,-0.387765,-1.229216,-0.091546,1.274274,1,-0.953914,0.188935,1.103202,0.157353,-0.761754,-1.142112
3,-0.541380,-0.764653,-0.815954,-0.498382,0,-1.059170,0.197710,1.103202,0.159915,0.222931,-0.831989
4,-0.541380,-0.764653,-0.815954,-0.498382,0,-1.059170,0.197710,-0.906452,0.156657,0.222931,-0.831989
...,...,...,...,...,...,...,...,...,...,...,...
3977,-0.457913,-1.297472,-0.091546,0.348257,0,-1.405518,0.189295,-0.906452,-0.719784,-0.761754,-1.281380
3978,-0.639681,-1.535574,-0.091546,-0.489871,1,-2.199202,0.201801,1.103202,0.160793,0.222931,-1.702106
3979,-0.639681,-1.535574,-0.091546,-0.489871,1,-2.199202,0.201801,-0.906452,0.160474,0.222931,-1.702106
3980,-0.639681,-1.535574,-0.091546,-0.489871,1,-2.199202,0.201801,-0.906452,0.160793,0.222931,-1.702106


In [363]:
import torch
import torch.nn as nn
import numpy as np

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize model
input_size = X_train.shape[1]  # number of features
model = ResidualMLP(input_size=input_size, num_classes=3).to(device)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005, weight_decay=1e-5)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.8, patience=6, min_lr=1e-6
)

# Early stopping setup
patience = 70
best_accuracy = 0.0
epochs_no_improve = 0
best_model_path = "K2_best_residual_mlp_model.pth"

# Loss function
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

# Training parameters
epochs = 500

for epoch in range(epochs):
    # --- Training ---
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    # --- Evaluation ---
    model.eval()
    test_loss, correct_test, total_test = 0.0, 0, 0
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = loss_fn(outputs, batch_y)
            test_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            total_test += batch_y.size(0)
            correct_test += (predicted == batch_y).sum().item()
    
    avg_test_loss = test_loss / len(test_loader)
    test_accuracy = 100 * correct_test / total_test

    # --- Scheduler step (manual print) ---
    old_lr = optimizer.param_groups[0]['lr']
    scheduler.step(test_accuracy)
    new_lr = optimizer.param_groups[0]['lr']
    if new_lr < old_lr:
        print(f"📉 Learning rate reduced from {old_lr:.6f} to {new_lr:.6f}")

    # --- Early stopping check ---
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(model.state_dict(), best_model_path)
        epochs_no_improve = 0
        print(f"✅ Saved new best model at epoch {epoch+1} | Test Acc: {best_accuracy:.2f}%")
    else:
        epochs_no_improve += 1
        print(f"⏳ No improvement for {epochs_no_improve}/{patience} epochs")
    
    if epochs_no_improve >= patience:
        print(f"🚨 Early stopping triggered at epoch {epoch+1}")
        print(f"🎯 Best Test Accuracy: {best_accuracy:.2f}%")
        model.load_state_dict(torch.load(best_model_path))
        break

    # --- Logging ---
    print(
        f"Epoch {epoch+1}/{epochs} | "
        f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
        f"Test Loss: {avg_test_loss:.4f}, Test Acc: {test_accuracy:.2f}%"
    )

print("\n🎯 Training finished.")
print(f"Best Test Accuracy: {best_accuracy:.2f}%")


✅ Saved new best model at epoch 1 | Test Acc: 90.86%
Epoch 1/500 | Train Loss: 0.6364, Train Acc: 80.14% | Test Loss: 0.5749, Test Acc: 90.86%
⏳ No improvement for 1/70 epochs
Epoch 2/500 | Train Loss: 0.4757, Train Acc: 85.10% | Test Loss: 0.4156, Test Acc: 90.06%
⏳ No improvement for 2/70 epochs
Epoch 3/500 | Train Loss: 0.4150, Train Acc: 87.74% | Test Loss: 0.3499, Test Acc: 88.76%
✅ Saved new best model at epoch 4 | Test Acc: 91.37%
Epoch 4/500 | Train Loss: 0.4065, Train Acc: 87.88% | Test Loss: 0.3775, Test Acc: 91.37%
⏳ No improvement for 1/70 epochs
Epoch 5/500 | Train Loss: 0.3673, Train Acc: 88.71% | Test Loss: 0.3402, Test Acc: 87.65%
✅ Saved new best model at epoch 6 | Test Acc: 94.18%
Epoch 6/500 | Train Loss: 0.3878, Train Acc: 87.24% | Test Loss: 0.4008, Test Acc: 94.18%
⏳ No improvement for 1/70 epochs
Epoch 7/500 | Train Loss: 0.3792, Train Acc: 89.55% | Test Loss: 0.3059, Test Acc: 92.37%
⏳ No improvement for 2/70 epochs
Epoch 8/500 | Train Loss: 0.3074, Train Acc: 9

In [364]:
from sklearn.metrics import classification_report

# --- Evaluate on test set and generate classification report ---
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        _, predicted = torch.max(outputs, 1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

# Generate report
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=4))



📊 Classification Report:
              precision    recall  f1-score   support

           0     0.9509    0.9012    0.9254       344
           1     0.9914    1.0000    0.9957       579
           2     0.6628    0.7808    0.7170        73

    accuracy                         0.9498       996
   macro avg     0.8684    0.8940    0.8794       996
weighted avg     0.9534    0.9498    0.9510       996



In [365]:
# Randomly sample 200 rows from the original dataframe for test set
df = pd.read_csv('Data\k2pandc final.csv')
test_sample_df = df.sample(n=200, random_state=42)

# Save to CSV
test_sample_df.to_csv("test_sample.csv", index=False)